載入必要套件與類別

In [1]:
import mplfinance as mpf
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
from grid_backtest import trade
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']

網格交易最佳化

In [2]:
# 取得要回測的歷史資料
symbol = '0050' 
data = yf.download(f'{symbol}.TW')
data1 = data.replace(0, None)
data = data1.ffill()
data.columns = [i.lower() for i in data.columns]

[*********************100%%**********************]  1 of 1 completed
C:\Users\User\AppData\Local\Temp\ipykernel_15768\1659100453.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data1.ffill()


In [3]:
# 紀錄最佳化績效
trade_performance = []
for i, j, k in [[i/100, j, k]
                for i in range(2, 11, 2)
                for j in [5, 20, 25]
                for k in [1.6, 1.8, 2, 2.2]]:
    # 設定初始部位(%數)
    init_ratio = 50
    # 設定網格間距
    grid_gap = round(i, 2)
    # 設定交易單位(%數)
    grid_unit = j
    # 初始部位是幾個交易單位
    if init_ratio % grid_unit != 0:
        print('錯誤！初始部位要可以被交易單位整除')
    init_unit = int(init_ratio / grid_unit)
    # 上漲跟下跌網格差距比率
    up_down_grid_gap_diff = k

    # 期初買入張數
    init_time = data.index[0]
    init_price = data.loc[data.index[0], 'open']
    trade_position = trade(init_time, init_price, init_unit)
    # 依照迴圈來跑網格
    for index, row in data.iterrows():
        # 優先執行比較差的狀況(賣出) 至少不會回測過度優化導致結果失真
        while row['open'] >= init_price * (1+grid_gap*up_down_grid_gap_diff):
            init_price *= (1+grid_gap*up_down_grid_gap_diff)
            trade_position.cover(index, row['open'])
        while row['high'] >= init_price * (1+grid_gap*up_down_grid_gap_diff):
            init_price *= (1+grid_gap*up_down_grid_gap_diff)
            trade_position.cover(index, init_price)
        # 再考慮(買進)的情況
        while row['open'] <= init_price * (1-grid_gap):
            init_price *= (1-grid_gap)
            if trade_position.position() < 100/grid_unit:
                trade_position.order(index, row['open'])
        while row['low'] <= init_price * (1-grid_gap):
            init_price *= (1-grid_gap)
            if trade_position.position() < 100/grid_unit:
                trade_position.order(index, init_price)

    # 最後一天的資料 要把所有部位先出場 檢查總績效
    while trade_position.position() > 0:
        trade_position.cover(index, row['close'])
    # 報酬率計算
    trade_position.restoreReturn(symbol)
    position_table = trade_position.position_table
    position_table['單筆報酬'] = (position_table['出場價格'] - position_table['進場價格'] +
                              position_table['還原除權息'])
    position_table['單筆報酬率'] = position_table['單筆報酬'] / \
        position_table['進場價格'] * (grid_unit/100)
    total_return = (position_table['單筆報酬率']).sum() + 1
    yearly_return = total_return ** (1/(data.shape[0]/252))-1
    # 紀錄最佳化參數與績效
    trade_performance.append([i, j, k, yearly_return])

    print(f'{i}, {j}, {k} 年化報酬率:{yearly_return}')

# 最佳化績效檢視
pdf = pd.DataFrame(trade_performance)

d:\pythonProject\台股網格自動化交易書\grid_backtest.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dividend_table_one["西元日期"] = pd.to_datetime(
d:\pythonProject\台股網格自動化交易書\grid_backtest.py:100: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.position_table.loc[index, "還原除權息"] += row1["權值+息值"]


0.08, 25, 1.8 年化報酬率:0.06400663406382412


d:\pythonProject\台股網格自動化交易書\grid_backtest.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dividend_table_one["西元日期"] = pd.to_datetime(
d:\pythonProject\台股網格自動化交易書\grid_backtest.py:100: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.position_table.loc[index, "還原除權息"] += row1["權值+息值"]


0.08, 25, 2 年化報酬率:0.0691316677665561


d:\pythonProject\台股網格自動化交易書\grid_backtest.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dividend_table_one["西元日期"] = pd.to_datetime(
d:\pythonProject\台股網格自動化交易書\grid_backtest.py:100: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.position_table.loc[index, "還原除權息"] += row1["權值+息值"]


...

d:\pythonProject\台股網格自動化交易書\grid_backtest.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dividend_table_one["西元日期"] = pd.to_datetime(
d:\pythonProject\台股網格自動化交易書\grid_backtest.py:100: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.position_table.loc[index, "還原除權息"] += row1["權值+息值"]


0.1, 25, 1.8 年化報酬率:0.07170432553125616


d:\pythonProject\台股網格自動化交易書\grid_backtest.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dividend_table_one["西元日期"] = pd.to_datetime(
d:\pythonProject\台股網格自動化交易書\grid_backtest.py:100: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.position_table.loc[index, "還原除權息"] += row1["權值+息值"]


0.1, 25, 2 年化報酬率:0.050839671467195924
0.1, 25, 2.2 年化報酬率:0.07440538419555698


d:\pythonProject\台股網格自動化交易書\grid_backtest.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dividend_table_one["西元日期"] = pd.to_datetime(
d:\pythonProject\台股網格自動化交易書\grid_backtest.py:100: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.position_table.loc[index, "還原除權息"] += row1["權值+息值"]


In [7]:
pdf.columns = ['網格間距', '交易單位', '上下網格差距比率', '年化報酬率']
pdf.sort_values(by='年化報酬率', ascending=False, inplace=True)
pdf

,網格間距,交易單位,上下網格差距比率,年化報酬率
39,0.08,5,2.2,0.099338
38,0.08,5,2.0,0.096877
37,0.08,5,1.8,0.095890
9,0.02,25,1.8,0.094901
8,0.02,25,1.6,0.093953
10,0.02,25,2.0,0.092275
36,0.08,5,1.6,0.092085
27,0.06,5,2.2,0.089348
5,0.02,20,1.8,0.088372
4,0.02,20,1.6,0.087899


: 